In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import network_builder
import constants 
import latent_semantic_analysis

# from recommender.core import network_builder

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/huynhhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/huynhhao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
companies_df = pd.read_csv(r'C:\Users\ASUS\Desktop\job_recommender\data\companies.csv')
jobs_df = pd.read_csv(r'C:\Users\ASUS\Desktop\job_recommender\data\jobs.csv')
cv = pd.read_csv(r'C:\Users\ASUS\Desktop\job_recommender\data\cvdata\ResumeDataSet.csv', encoding = 'utf-8')

In [39]:
# transform companies_df and jobs_df into dicts
employers_data = {}
jobs_data = {}
for i, row in companies_df.iterrows():
    employer_data = {'company_name': row['company_name'],
                   'average_rating': row['average_rating'],
                   'num_review': row['num_review'],
                   'city': row['city'],
                   'type': row['type'],
                   'num_employee': row['num_employee'],
                   'country': row['country'],
                   'working_day': row['working_day'],
                   'OT': row['OT'],
                   'overview': row['overview'],
                   'expertise': row['expertise'],
                   'benifit': row['benifit'],
                   'logo_link': row['logo_link']}
    
    employers_data[row['company_id']] = employer_data
    
for i, row in jobs_df.iterrows():
    job_data = {'company_id': row['company_id'],
               'job_name': row['job_name'],
               'taglist': row['taglist'],
               'location': row['location'], 
               'three_reasons': row['three_reasons'],
               'description': row['description']}
    jobs_data[row['job_id']] = job_data
    

In [68]:
G = nx.MultiDiGraph(name = 'Jobs graph')
# first add all employer nodes to the network and its data
for employer_id, employer_data in employers_data.items():
    G.add_node(employer_id, node_type = 'employer', **employer_data)
    
# add all job nodes and the bidirectional edges from job node to employer node
for job_id, job_data in jobs_data.items():
    G.add_node(job_id, node_type = 'job', **job_data)
    # add two edges between job and its employers\
    G.add_edge(job_id, job_data['company_id'], weight = 1)
    G.add_edge(job_data['company_id'], job_id, weight = 1)

In [74]:
G['kms-technology']

AdjacencyView({'kms-technology:jrsr_qa_engineer_kms_labs_bonus': {0: {'weight': 1}}, 'kms-technology:engineering_manager_bonus': {0: {'weight': 1}}, 'kms-technology:fullstack_mobile_mobilenodejs_kobiton': {0: {'weight': 1}}, 'kms-technology:jrsrprincipal_java_developer_bonus': {0: {'weight': 1}}, 'kms-technology:product_manager_kms_labs_bonus': {0: {'weight': 1}}, 'kms-technology:sr_it_business_analyst_english_bonus': {0: {'weight': 1}}, 'kms-technology:fullstack_dev_reactjsnodejs_kobiton': {0: {'weight': 1}}, 'kms-technology:senior_ruby_on_rails_engineer_bonus': {0: {'weight': 1}}, 'kms-technology:senior_data_engineer_bonus': {0: {'weight': 1}}, 'kms-technology:srjr_fullstack_nodejsreactjs_bonus': {0: {'weight': 1}}, 'kms-technology:juniorsenior_test_engineer_bonus': {0: {'weight': 1}}, 'kms-technology:jrsrprincipal_net_developer_bonus': {0: {'weight': 1}}})

In [3]:
nb = network_builder.NetworkBuilder(constants.EMPLOYERS_DATA_PATH,
                                    constants.JOBS_DATA_PATH, constants.CV_DATAPATH )

In [5]:
nb.build()

2021-11-26 06:47:00,391 - Start building the master Graph...
2021-11-26 06:47:00,418 - Master Graph is built.
2021-11-26 06:47:00,419 - Creating LSA comparer
2021-11-26 06:47:00,420 - Loading LSA comparer from /home/huynhhao/Desktop/job_recommender/data/network_data/lsa.pkl
2021-11-26 06:47:00,518 - Adding relations edges...


In [8]:
len(nb.G.nodes)

4414

In [54]:
nb.G.nodes['kms-technology']

{'node_type': 'employer',
 'company_name': 'KMS Technology',
 'average_rating': 4.0,
 'num_review': '197 Reviews',
 'city': 'Ho Chi Minh, Da Nang, Ha Noi',
 'type': 'Outsourcing',
 'num_employee': '1000+',
 'country': 'United States',
 'working_day': 'Monday - Friday',
 'OT': 'No OT',
 'overview': 'Established in 2009, KMS Technology is a U.S.-based engineering and services company with development centers in Vietnam. KMS Technology is trusted by international clients for the superior quality of products and expertise of Vietnamese engineers.\n\nKMS builds and successfully launches its own software companies through its internal startup incubator,\xa0KMS Labs. Most notable companies include\xa0QASymphony,\xa0Kobiton,\xa0Katalon, and\xa0Grove. Under the brand,\xa0KMS Solutions, the company serves the Asia Pacific region, offering technology solution consulting and bringing the most advanced and latest technologies to the Asian market.\xa0\n\nKMS is committed to making a long-lasting soc

In [53]:
nb.G.nodes['hansen-technologies:manual_test_engineer_testerqa_qc']

{'node_type': 'job',
 'company_id': 'hansen-technologies',
 'job_name': 'Manual Test Engineer (Tester/QA QC)',
 'taglist': ' Tester   English   QA QC ',
 'location': '98 Xuan Thuy Street, Thao Dien Ward, District 2, Ho Chi Minh',
 'three_reasons': 'Global Company- Develop Your Career & English\nCompetitive Salary, and company profit share\n13th salary as sign-on bonus (1 Oct. 2021 onward)',
 'description': 'Why work for us?\n\n\nEvery day, we at Hansen inspire and challenge today’s utilities and telecoms to be tomorrow’s next experience companies.\n\nFrom North America to Australia and everywhere in between, you’ll be part of a global team of technology professionals who work to enable our clients to innovate more freely and engage at a new level with their customers\n\n\nWho are Hansen Technologies?\n\n\nWe are a global provider of software and services to the energy, water and communications industries. With our award-winning software suite, we help more than 600 customers in over 80

In [31]:
nb.lsa.preprocess_text(nb.G.nodes['kms-technology']['overview'])

'established km technology usbased engineering service company development center vietnam km technology trusted international client superior quality product expertise vietnamese engineer km build successfully launch software company internal startup _unknown_ km lab _unknown_ company include qasymphony kobiton katalon _unknown_ brand km solution company serf asia pacific region offering technology solution consulting bringing advanced latest technology asian market km committed making _unknown_ social impact partnering _unknown_ organization give _unknown_ student fair better employment opportunity company regularly _unknown_ university participates training activity specialized event sponsor scholarship program vietnam company received many prestigious industry award named one greatest place work vietnam asia u many year row'

In [49]:
nb.G.nodes

NodeView(('kms-technology', 'nab-in-collaboration-with-positive-thinking-company-in-vietnam', 'robert-bosch-engineering-and-business-solutions', 'gft-technologies-vietnam', 'tyme', 'hansen-technologies', 'bbv-vietnam', 'dek-technologies', 'money-forward-vietnam-co-ltd', 'titan-technology-corporation', 'nfq-asia', 'vietnam-international-bank-vib', 'm_service-momo', 'dirox', 'be-group', 'nvg-technology', 'propzy', 'skedulo-vietnam', 'ngan-hang-a-chau-acb', 'scc-vietnam', 'vng-corporation', 'niteco-vietnam-co-ltd', 'fpt-software', 'ominext', 'zalo', 'contemi-vietnam', 'capgemini-vietnam', 'diag', 'samsung-electronics-hcmc-ce-complex', 'nakivo', 'saigon-technology-solutions', 'doctor-anywhere-viet-nam', 'viettel-group', 'hdwebsoft', 'deliveree-on-demand-logistics', 'vien-nghien-cuu-du-lieu-lon-vinbigdata', 'chotot', 'live-group', 'east-agile', 'wonderlabs', 'wata', 'taptap', 'line-vietnam-2', 'hdbank', 'parcel-perform', 'go2joy', 'rmit-university-vietnam', 'houze-group', 'vietjetair-cargo-